In [ ]:
from src.data_collection.data_collection import (CDGClient, API_VERSION, ROOT_URL, RESPONSE_FORMAT, 
                                                 RESULT_LIMIT, RATE_LIMIT_CONSTANT, gather_congressional_records)
from src.data_structures.bills import (
    Format,
    BehalfType,
    AmendmentType,
    TextVersion,
    PolicyArea,
    LegislativeSubject,
    Chamber,
    LawType,
    LatestAction,
    Note,
    Summary,
    CommitteeMetadata,
    SourceSystem,
    ActionSourceSystem,
    Activity,
    IdentifyingEntity,
    RelationshipDetail,
    CountUrl,
    Title,
    ChamberCode,
    BillType,
    Member,
    Sponsor,
    LawMetadata,
    Law,
    Committee,
    RecordedVote,
    Action,
    AmendmentMetadata,
    Treaty,
    Amendment,
    BillMetadata,
    Subjects,
    Bill
)

import os 
import json 

bills_file = [x for x in os.listdir() if 'bills_2025' in x][0]
additional_data_file = [x for x in os.listdir() if 'additional_data' in x][0]

with open(bills_file, 'r') as f:
    bills = json.load(f)

with open(additional_data_file, 'r') as f:
    additional_data = json.load(f)

def save_records(records, filename):
    with open(filename, 'w') as f:
        json.dump(records, f)

def load_records(filename):
    with open(filename, 'r') as f:
        return json.load(f)
    
records = load_records('congressional_records.json')

import pickle
def pickle_data(data, filename):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)

def unpickle_data(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

parsed_records = unpickle_data('records_parsed.pkl')

In [2]:
import os 
client = CDGClient(api_key=os.environ["CONGRESS_API_KEY"], response_format=RESPONSE_FORMAT)


In [6]:
members = client.get_members(params={'limit': 250})

TypeError: CDGClient.get_members() got an unexpected keyword argument 'params'

In [5]:
len(members)

3

In [ ]:
class Member(BaseModel):
    bioguideId: str
    depiction: Depiction
    district: Optional[str]  # district can be None, so it's optional
    name: str
    partyName: str
    state: str
    terms: List[Term]
    updateDate: str
    url: HttpUrl